<a href="https://colab.research.google.com/github/aleksandarb99/profession-and-gender-recognition/blob/develop/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("cao")

cao


In [ ]:
from __future__ import print_function

import os
from io import open
import requests
import shutil
import numpy as np
import json
from IPython.display import Image
from zipfile import ZipFile
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Conv2D, Activation, Dropout, Flatten, Dense, MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras.preprocessing import image

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
execution_path = os.getcwd()
print(execution_path)
DATASET_DIR = os.path.join(execution_path, "idenprof")
DATASET_TRAIN_DIR = os.path.join(DATASET_DIR, "train")
DATASET_TEST_DIR = os.path.join(DATASET_DIR, "test")

!ls "/content/drive/profession-detection-master"

/content
ls: cannot access '/content/drive/profession-detection-master': No such file or directory


In [ ]:
save_direc = os.path.join(os.getcwd(), 'idenprof_models')

# Name of model files
model_name = 'idenprof_weight_model_VGG19.{epoch:03d}-{val_acc}.h5'

# Create Directory if it doesn't exist
if not os.path.isdir(save_direc):
    os.makedirs(save_direc)
# Join the directory with the model file
modelpath = os.path.join(save_direc, model_name)

# Checkpoint to save best model
checkpoint = ModelCheckpoint(filepath = modelpath, monitor = 'val_acc', verbose = 1, save_best_only = True,
                             save_weights_only = True, period=1)

In [ ]:
input_shape = (224, 224, 3)

model = Sequential(
    [Conv2D(64, (3, 3), input_shape=input_shape, padding='same',activation='relu'),
     Conv2D(64, (3, 3), activation='relu', padding='same'),
     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
     Conv2D(128, (3, 3), activation='relu', padding='same'),
     Conv2D(128, (3, 3), activation='relu', padding='same',),
     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
     Conv2D(256, (3, 3), activation='relu', padding='same',),
     Conv2D(256, (3, 3), activation='relu', padding='same',),
     Conv2D(256, (3, 3), activation='relu', padding='same',),
     Conv2D(256, (3, 3), activation='relu', padding='same',),
     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
     Conv2D(512, (3, 3), activation='relu', padding='same',),
     Conv2D(512, (3, 3), activation='relu', padding='same',),
     Conv2D(512, (3, 3), activation='relu', padding='same',),
     Conv2D(512, (3, 3), activation='relu', padding='same',),
     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
     Conv2D(512, (3, 3), activation='relu', padding='same',),
     Conv2D(512, (3, 3), activation='relu', padding='same',),
     Conv2D(512, (3, 3), activation='relu', padding='same',),
     Conv2D(512, (3, 3), activation='relu', padding='same',),
     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
     Flatten(),
     Dense(10, activation="softmax")])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

In [ ]:
def train_network(model):
    print(os.listdir(os.path.join(execution_path, "idenprof")))
    optimizer = keras.optimizers.Adam(lr=0.01, decay=1e-4)
    batch_size = 32
    num_classes = 10
    epochs = 200
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    print("Using real time Data Augmentation")
    train_datagen = ImageDataGenerator(rescale=1. / 255,horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(DATASET_TRAIN_DIR, target_size=(224, 224),batch_size=batch_size,
                                                        class_mode="categorical")
    test_generator = test_datagen.flow_from_directory(DATASET_TEST_DIR, target_size=(224, 224), batch_size=batch_size,
                                                      class_mode="categorical")
    model.fit_generator(train_generator, steps_per_epoch=int(9000 / batch_size), epochs=epochs,
                        validation_data=test_generator,validation_steps=int(2000 / batch_size),
                        callbacks=[checkpoint])

In [ ]:
train_network(model)

FileNotFoundError: ignored

In [ ]:
CLASS_INDEX = None
MODEL_PATH = os.path.join(execution_path, "idenprof_VGG19_053-0.726.h5")
JSON_PATH = os.path.join(execution_path, "idenprof_model_class.json")

In [ ]:
model.load_weights(MODEL_PATH)

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.01, decay=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(DATASET_TEST_DIR, target_size=(224, 224), batch_size = 32,
                                                  class_mode="categorical")
loss,accuracy = model.evaluate_generator(test_generator,steps=int(2000 / 32))
print("Accuracy =", accuracy)

In [ ]:
def preprocess_input(x):
    x *= (1. / 255)
    return x

In [ ]:
def decode_predictions(preds, top=5, model_json=""):
    global CLASS_INDEX
    if CLASS_INDEX is None:
        CLASS_INDEX = json.load(open(model_json))
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        for i in top_indices:
            each_result = []
            each_result.append(CLASS_INDEX[str(i)])
            each_result.append(pred[i])
            results.append(each_result)
    return results

In [ ]:
def run_inference(model,picture):
    image_to_predict = image.load_img(picture, target_size=(224, 224))
    image_to_predict = image.img_to_array(image_to_predict, data_format="channels_last")
    image_to_predict = np.expand_dims(image_to_predict, axis=0)

    image_to_predict = preprocess_input(image_to_predict)

    prediction = model.predict(x=image_to_predict, steps=1)

    predictiondata = decode_predictions(prediction, top=int(5), model_json=JSON_PATH)

    for result in predictiondata:
        print(str(result[0]), " : ", str(result[1] * 100))

In [ ]:
picture = os.path.join(execution_path, "test-images/1.jpg")
display(Image(filename=picture))
run_inference(model,picture)

In [ ]:
picture = os.path.join(execution_path, "test-images/2.jpg")
display(Image(filename=picture))
run_inference(model,picture)

In [ ]:
picture = os.path.join(execution_path, "test-images/3.jpg")
display(Image(filename=picture))
run_inference(model,picture)